In [100]:
#Importing libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from os import getcwd
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Embedding, Dropout
from sklearn import preprocessing

In [101]:
#Reading data
url = "https://hub.mph.in.gov/dataset/bd08cdd3-9ab1-4d70-b933-41f9ef7b809d/resource/afaa225d-ac4e-4e80-9190-f6800c366b58/download/covid_report_county_date.xlsx?raw=true"
cases_data = pd.read_excel(url)

population_data = pd.read_csv(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/indiana countywise population.csv")

#Data preprocessing
population_data["County"] = population_data["County"].str.upper()
population_data["County"] = population_data["County"].str.replace('.', '')
population_data["County"] = population_data["County"].str.replace(' ', '')
cases_data["COUNTY_NAME"] = cases_data["COUNTY_NAME"].str.replace(' ','')

data = pd.merge(cases_data, population_data, how = 'inner', left_on = 'COUNTY_NAME', right_on = 'County')
data = data[["COUNTY_NAME", "DATE", "COVID_COUNT",  "COVID_DEATHS", "COVID_TEST", "Population"]]
data

COUNTY_NAME        DATE  ...  COVID_TEST  Population
0           ADAMS  2020-02-29  ...           0       35195
1           ADAMS  2020-03-02  ...           0       35195
2           ADAMS  2020-03-03  ...           0       35195
3           ADAMS  2020-03-04  ...           0       35195
4           ADAMS  2020-03-05  ...           1       35195
...           ...         ...  ...         ...         ...
11403     WHITLEY  2020-06-28  ...          11       33649
11404     WHITLEY  2020-06-29  ...          22       33649
11405     WHITLEY  2020-06-30  ...          23       33649
11406     WHITLEY  2020-07-01  ...           1       33649
11407     WHITLEY  2020-07-02  ...           0       33649

[11408 rows x 6 columns]

In [102]:
data.describe()

COVID_COUNT  COVID_DEATHS    COVID_TEST     Population
count  11408.000000  11408.000000  11408.000000   11408.000000
mean       4.112465      0.218093     44.192935   72145.934783
std       15.137558      0.924302    115.533138  120873.476617
min        0.000000      0.000000      0.000000    5887.000000
25%        0.000000      0.000000      3.000000   20856.500000
50%        0.000000      0.000000     11.000000   34061.500000
75%        2.000000      0.000000     35.000000   67639.750000
max      432.000000     16.000000   1734.000000  944523.000000

In [103]:
county_name = data["COUNTY_NAME"].unique()
county_name

array(['ADAMS', 'ALLEN', 'BARTHOLOMEW', 'BENTON', 'BLACKFORD', 'BOONE',
       'BROWN', 'CARROLL', 'CASS', 'CLARK', 'CLAY', 'CLINTON', 'CRAWFORD',
       'DAVIESS', 'DEARBORN', 'DECATUR', 'DEKALB', 'DELAWARE', 'DUBOIS',
       'ELKHART', 'FAYETTE', 'FLOYD', 'FOUNTAIN', 'FRANKLIN', 'FULTON',
       'GIBSON', 'GRANT', 'GREENE', 'HAMILTON', 'HANCOCK', 'HARRISON',
       'HENDRICKS', 'HENRY', 'HOWARD', 'HUNTINGTON', 'JACKSON', 'JASPER',
       'JAY', 'JEFFERSON', 'JENNINGS', 'JOHNSON', 'KNOX', 'KOSCIUSKO',
       'LAGRANGE', 'LAKE', 'LAPORTE', 'LAWRENCE', 'MADISON', 'MARION',
       'MARSHALL', 'MARTIN', 'MIAMI', 'MONROE', 'MONTGOMERY', 'MORGAN',
       'NEWTON', 'NOBLE', 'OHIO', 'ORANGE', 'OWEN', 'PARKE', 'PERRY',
       'PIKE', 'PORTER', 'POSEY', 'PULASKI', 'PUTNAM', 'RANDOLPH',
       'RIPLEY', 'RUSH', 'SCOTT', 'SHELBY', 'SPENCER', 'STJOSEPH',
       'STARKE', 'STEUBEN', 'SULLIVAN', 'SWITZERLAND', 'TIPPECANOE',
       'TIPTON', 'UNION', 'VANDERBURGH', 'VERMILLION', 'VIGO', 'WABASH',
   

In [104]:
print('Length of the sample: ', len(data))

Length of the sample:  11408


In [105]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [106]:
#Scaling the data
scaler = preprocessing.StandardScaler()
scaled_data = scaler.fit_transform(data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population']])

data["COVID_COUNT"] = scaled_data[:, 0]
data["COVID_DEATHS"] = scaled_data[:, 1]
data["COVID_TEST"] = scaled_data[:, 2]
data["Population"] = scaled_data[:, 3]

data

COUNTY_NAME        DATE  ...  COVID_TEST  Population
0           ADAMS  2020-02-29  ...   -0.382530   -0.305713
1           ADAMS  2020-03-02  ...   -0.382530   -0.305713
2           ADAMS  2020-03-03  ...   -0.382530   -0.305713
3           ADAMS  2020-03-04  ...   -0.382530   -0.305713
4           ADAMS  2020-03-05  ...   -0.373874   -0.305713
...           ...         ...  ...         ...         ...
11403     WHITLEY  2020-06-28  ...   -0.287315   -0.318503
11404     WHITLEY  2020-06-29  ...   -0.192100   -0.318503
11405     WHITLEY  2020-06-30  ...   -0.183444   -0.318503
11406     WHITLEY  2020-07-01  ...   -0.373874   -0.318503
11407     WHITLEY  2020-07-02  ...   -0.382530   -0.318503

[11408 rows x 6 columns]

In [107]:
# reshape to [samples, time_steps, n_features]
time_steps = 21
X_train, y_train = [], []

for county in county_name:
  filtered_data = data.loc[data["COUNTY_NAME"] == county]
  train_size = len(filtered_data)
  filtered_train = filtered_data.iloc[0:train_size]
  
  filtered_X_train, filtered_y_train = create_dataset(filtered_train[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population']], 
                                                      filtered_train[['COVID_COUNT', 'COVID_DEATHS']], time_steps)

  if(len(X_train) == 0):
    X_train, y_train = filtered_X_train, filtered_y_train
  else:
    X_train = np.vstack((X_train, filtered_X_train))
    y_train = np.vstack((y_train, filtered_y_train))

print(X_train.shape, y_train.shape)

(9476, 21, 4) (9476, 2)


In [95]:
# Loading model
model = keras.models.load_model(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/checkpoint.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 1024)          4214784   
_________________________________________________________________
dropout (Dropout)            (None, 10, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 512)           3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           787456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                8

In [108]:
# #Defining Model
# model = Sequential()
# model.add(LSTM(1024, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.1))
# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dropout(0.1))
# model.add(Dense(32))
# model.add(Dense(2))

# model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 1024)          4214784   
_________________________________________________________________
dropout (Dropout)            (None, 21, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 21, 512)           3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 21, 256)           787456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                8

In [112]:
model.compile(loss = 'mean_squared_error', optimizer = 'rmsprop')

history = model.fit(
    X_train, y_train,
    epochs = 500
)

Epoch 1/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0255
Epoch 2/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0332
Epoch 3/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0262
Epoch 4/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0228
Epoch 5/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0226
Epoch 6/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0275
Epoch 7/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0236
Epoch 8/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0245
Epoch 9/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0239
Epoch 10/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0222
Epoch 11/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0198
Epoch 12/500
297/297 [==============================] - 6s 21ms/step - loss: 0.0224
E

In [115]:
checkpoint_path = f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/checkpoint.h5"
model.save(checkpoint_path)

In [116]:
import datetime as dt

for county in county_name:
  demo_data = data[data['COUNTY_NAME'] == county]
  demo_data = demo_data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population']]
  demo_data = demo_data.tail(time_steps + 1)

  demo_X_test, demo_y_test = create_dataset(demo_data[['COVID_COUNT', 'COVID_DEATHS', 'COVID_TEST', 'Population']], 
                                                      demo_data[['COVID_COUNT', 'COVID_DEATHS']], time_steps)

  predictions = np.concatenate((model.predict(demo_X_test), np.zeros((1, 2))), axis = 1)
  predictions = np.intc(scaler.inverse_transform(predictions))
  # print(county, ':', predictions[:, 0:2])

  transformed_demo_data = np.intc(scaler.inverse_transform(demo_data))
  demo_data['COVID_COUNT'] = transformed_demo_data[:, 0]
  demo_data['COVID_DEATHS'] = transformed_demo_data[:, 1]
  demo_data = demo_data[['COVID_COUNT', 'COVID_DEATHS']]
  demo_data = np.concatenate((demo_data, predictions[:, 0:2]), axis = 0)

  date_data = data[['DATE']]
  date_data = date_data.tail(time_steps + 2)
  date_data['DATE'] = pd.to_datetime(date_data['DATE']).apply(pd.DateOffset(1))

  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.suptitle('COVID FORECASTING for ' + county + ' County')
  fig.set_size_inches(15, 6)
  plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
  plt.setp(ax2.xaxis.get_majorticklabels(), rotation=90)

  ax1.plot(date_data, demo_data[:, 0], linestyle='-', marker='o', color = 'red', label = 'COVID CASES')
  ax1.grid(True)
  ax1.set_xlabel('Time')
  ax1.set_ylabel('Count')
  ax1.legend()

  ax2.plot(date_data, demo_data[:, 1], linestyle='-', marker='o', color = 'blue', label = 'COVID DEATHS')
  ax2.grid(True)
  ax2.set_xlabel('Time')
  ax2.set_ylabel('Count')
  ax2.legend()
  # plt.show()

  fig.savefig(f"{getcwd()}/drive/My Drive/Colab Notebooks/COVID forecasting/output images/"+county+".png", bbox_inches='tight')
  plt.close()